In [1]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🐺 COMBINED SYSTEM BACKTEST")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("Mission: Test if combining all signals creates a tradeable edge")
print("="*60)

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🐺 COMBINED SYSTEM BACKTEST
Date: 2026-01-06 23:01
Mission: Test if combining all signals creates a tradeable edge


In [2]:
# UNIVERSE AND SECTOR CHAINS
UNIVERSE = {
    'Space': ['SIDU', 'ASTS', 'LUNR', 'RKLB', 'RDW'],
    'Quantum': ['IONQ', 'QBTS', 'RGTI', 'QUBT'],
    'Semiconductor': ['NVTS', 'WOLF', 'ON', 'AEHR', 'SKYT'],
    'Nuclear': ['UUUU', 'LEU', 'CCJ', 'SMR', 'OKLO'],
    'Rare_Earth': ['USAR', 'MP']
}

SECTOR_LEADERS = {
    'Space': 'RKLB',
    'Quantum': 'IONQ',
    'Semiconductor': 'NVDA',  # External leader
    'Nuclear': 'CCJ',
    'Rare_Earth': 'MP'
}

ALL_TICKERS = [t for sector in UNIVERSE.values() for t in sector]
print(f"📊 Testing {len(ALL_TICKERS)} stocks across {len(UNIVERSE)} sectors")

📊 Testing 21 stocks across 5 sectors


In [3]:
def get_stock_data(ticker, lookback_months=6):
    """Get comprehensive stock data for analysis."""
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_months * 30)
        
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if len(hist) < 20:
            return None
        
        # Core calculations
        hist['prev_close'] = hist['Close'].shift(1)
        hist['pct_change'] = ((hist['Close'] - hist['prev_close']) / hist['prev_close']) * 100
        hist['gap_pct'] = ((hist['Open'] - hist['prev_close']) / hist['prev_close']) * 100
        hist['vol_ratio'] = hist['Volume'] / hist['Volume'].rolling(20).mean()
        hist['is_green'] = hist['Close'] > hist['Open']
        hist['is_red'] = hist['Close'] < hist['Open']
        
        # RSI for leader exhaustion
        delta = hist['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        hist['rsi'] = 100 - (100 / (1 + rs))
        
        return hist
    except:
        return None

print("✅ get_stock_data() ready")

✅ get_stock_data() ready


In [4]:
# SIGNAL CHECKERS

def check_serial_runner(ticker, hist, min_runs=3):
    """
    Check if stock is a serial runner (has 3+ runs in lookback period).
    Returns (is_serial, run_count, avg_gain, avg_length)
    """
    runs = []
    i = 0
    
    while i < len(hist) - 3:
        if hist['pct_change'].iloc[i] > 0:  # Green day
            run_length = 1
            j = i + 1
            
            while j < len(hist) and hist['pct_change'].iloc[j] > 0:
                run_length += 1
                j += 1
            
            if run_length >= 3:
                start_price = hist['prev_close'].iloc[i] if i > 0 else hist['Open'].iloc[i]
                end_price = hist['Close'].iloc[j-1]
                if pd.notna(start_price) and start_price > 0:
                    gain = ((end_price / start_price) - 1) * 100
                    if gain >= 10:
                        runs.append({'length': run_length, 'gain': gain})
                
                i = j
                continue
        i += 1
    
    if len(runs) >= min_runs:
        avg_gain = np.mean([r['gain'] for r in runs])
        avg_length = np.mean([r['length'] for r in runs])
        return (True, len(runs), avg_gain, avg_length)
    
    return (False, len(runs), 0, 0)

def check_day0(hist, idx):
    """
    Check if given day is a Day 0 setup (day before potential run).
    Day 0 signals:
    - Red or flat day (consolidation)
    - Lower volume than recent average
    - After 1-3 red days (rest period)
    """
    if idx < 3:
        return False
    
    # Current day should be red or flat
    current_red = hist['pct_change'].iloc[idx] <= 0.5
    
    # Lower volume (resting)
    low_volume = hist['vol_ratio'].iloc[idx] < 1.2 if pd.notna(hist['vol_ratio'].iloc[idx]) else True
    
    # After at least 1 red day
    prev_red = hist['pct_change'].iloc[idx-1] < 0
    
    return current_red and low_volume and prev_red

def check_leader_exhaustion(leader_hist, idx):
    """
    Check if sector leader is showing exhaustion.
    - RSI > 65 (overbought)
    - At or near local high
    - Starting to decline
    """
    if idx < 10 or idx >= len(leader_hist):
        return False
    
    rsi = leader_hist['rsi'].iloc[idx]
    if pd.isna(rsi):
        return False
    
    # RSI overbought or recently was
    rsi_high = rsi > 60 or leader_hist['rsi'].iloc[idx-1:idx+1].max() > 65
    
    # Starting to decline
    declining = leader_hist['pct_change'].iloc[idx] < 0
    
    return rsi_high and declining

def check_premarket(hist, idx):
    """
    Check if day shows pre-market strength (gap up + volume).
    - Gap up 2%+
    - Volume 1.3x+ normal
    """
    if idx >= len(hist):
        return False
    
    gap = hist['gap_pct'].iloc[idx]
    vol = hist['vol_ratio'].iloc[idx]
    
    if pd.isna(gap) or pd.isna(vol):
        return False
    
    return gap >= 2.0 and vol >= 1.3

print("✅ Signal checkers ready")

✅ Signal checkers ready


In [5]:
# FIND ALL TRADE OPPORTUNITIES
print("\n" + "="*60)
print("🔍 SCANNING FOR TRADE OPPORTUNITIES")
print("="*60)

# Load all data first
stock_data = {}
serial_runners = {}
leader_data = {}

print("\n📊 Loading stock data...")
for ticker in ALL_TICKERS:
    hist = get_stock_data(ticker)
    if hist is not None:
        stock_data[ticker] = hist
        is_serial, run_count, avg_gain, avg_length = check_serial_runner(ticker, hist)
        if is_serial:
            serial_runners[ticker] = {
                'run_count': run_count,
                'avg_gain': avg_gain,
                'avg_length': avg_length
            }

print(f"   Loaded: {len(stock_data)} stocks")
print(f"   Serial runners identified: {len(serial_runners)}")

# Load leader data
for sector, leader in SECTOR_LEADERS.items():
    hist = get_stock_data(leader)
    if hist is not None:
        leader_data[sector] = hist


🔍 SCANNING FOR TRADE OPPORTUNITIES

📊 Loading stock data...
   Loaded: 21 stocks
   Serial runners identified: 19


In [6]:
# FIND OPPORTUNITIES WITH SIGNAL COUNTS
print("\n" + "="*60)
print("🎯 FINDING TRADE OPPORTUNITIES")
print("="*60)

opportunities = []

# Get sector for each ticker
ticker_sector = {}
for sector, tickers in UNIVERSE.items():
    for ticker in tickers:
        ticker_sector[ticker] = sector

# Scan each serial runner for entry opportunities
for ticker in serial_runners:
    if ticker not in stock_data:
        continue
    
    hist = stock_data[ticker]
    sector = ticker_sector.get(ticker)
    
    # Scan each day looking for Day 0 + confirmation next day
    for i in range(10, len(hist) - 5):  # Leave room to see results
        # Check Day 0
        is_day0 = check_day0(hist, i)
        
        if not is_day0:
            continue
        
        # Check pre-market (next day)
        has_premarket = check_premarket(hist, i+1)
        
        # Check leader exhaustion
        leader_exhausted = False
        if sector in leader_data:
            lhist = leader_data[sector]
            # Find matching date in leader data
            date = hist.index[i]
            if date in lhist.index:
                lidx = lhist.index.get_loc(date)
                leader_exhausted = check_leader_exhaustion(lhist, lidx)
        
        # Count signals
        signals = {
            'serial_runner': True,  # Already filtered for this
            'day0': is_day0,
            'premarket': has_premarket,
            'leader_exhausted': leader_exhausted
        }
        signal_count = sum(signals.values())
        
        # Calculate what happened (for backtesting)
        entry_date = hist.index[i+1]
        entry_price = hist['Open'].iloc[i+1]
        
        # Check next 5 days for outcome
        best_gain = 0
        exit_day = 0
        for j in range(1, 6):
            if i+1+j < len(hist):
                day_close = hist['Close'].iloc[i+1+j]
                gain = ((day_close / entry_price) - 1) * 100
                if gain > best_gain:
                    best_gain = gain
                    exit_day = j
        
        # Is this a successful trade?
        is_winner = best_gain >= 5  # 5% threshold for "success"
        
        opportunities.append({
            'ticker': ticker,
            'sector': sector,
            'entry_date': entry_date,
            'entry_price': entry_price,
            'signal_count': signal_count,
            'signals': signals,
            'best_gain': best_gain,
            'exit_day': exit_day,
            'is_winner': is_winner
        })

print(f"\n📊 Found {len(opportunities)} potential trade setups")


🎯 FINDING TRADE OPPORTUNITIES

📊 Found 548 potential trade setups


In [7]:
# ANALYZE BY SIGNAL COUNT
print("\n" + "="*60)
print("📊 WIN RATE BY SIGNAL COUNT")
print("="*60)

if opportunities:
    df = pd.DataFrame(opportunities)
    
    print(f"\n{'Signals':>10} {'Trades':>10} {'Winners':>10} {'Win Rate':>10} {'Avg Gain':>12}")
    print("-" * 55)
    
    for signal_count in sorted(df['signal_count'].unique()):
        subset = df[df['signal_count'] == signal_count]
        winners = subset['is_winner'].sum()
        win_rate = winners / len(subset) * 100 if len(subset) > 0 else 0
        avg_gain = subset['best_gain'].mean()
        
        print(f"{signal_count:>10} {len(subset):>10} {winners:>10} {win_rate:>9.0f}% {avg_gain:>11.1f}%")
    
    # Overall stats
    print("-" * 55)
    total_winners = df['is_winner'].sum()
    total_win_rate = total_winners / len(df) * 100
    total_avg = df['best_gain'].mean()
    print(f"{'TOTAL':>10} {len(df):>10} {total_winners:>10} {total_win_rate:>9.0f}% {total_avg:>11.1f}%")
else:
    print("❌ No trade opportunities found")


📊 WIN RATE BY SIGNAL COUNT

   Signals     Trades    Winners   Win Rate     Avg Gain
-------------------------------------------------------
         2        448        200        45%         7.1%
         3         97         60        62%        17.7%
         4          3          2        67%        23.7%
-------------------------------------------------------
     TOTAL        548        262        48%         9.0%


In [8]:
# KEY FINDING: Does more signals = better results?
print("\n" + "="*60)
print("📊 SIGNAL COUNT CORRELATION")
print("="*60)

if len(df) > 10:
    # Compare high signal (3+) vs low signal (1-2)
    high_signal = df[df['signal_count'] >= 3]
    low_signal = df[df['signal_count'] <= 2]
    
    print(f"\n📈 HIGH SIGNAL (3+ signals):")
    print(f"   Trades: {len(high_signal)}")
    if len(high_signal) > 0:
        print(f"   Win rate: {high_signal['is_winner'].mean()*100:.0f}%")
        print(f"   Avg gain: {high_signal['best_gain'].mean():.1f}%")
    
    print(f"\n📉 LOW SIGNAL (1-2 signals):")
    print(f"   Trades: {len(low_signal)}")
    if len(low_signal) > 0:
        print(f"   Win rate: {low_signal['is_winner'].mean()*100:.0f}%")
        print(f"   Avg gain: {low_signal['best_gain'].mean():.1f}%")
    
    # Correlation
    corr_win = df['signal_count'].corr(df['is_winner'].astype(int))
    corr_gain = df['signal_count'].corr(df['best_gain'])
    
    print(f"\n📊 CORRELATIONS:")
    print(f"   Signal count → Win rate: {corr_win:.2f}")
    print(f"   Signal count → Gain: {corr_gain:.2f}")
    
    if corr_win > 0.15 and corr_gain > 0.15:
        print(f"\n   ✅ MORE SIGNALS = BETTER OUTCOMES")
    elif corr_win > 0 or corr_gain > 0:
        print(f"\n   ⚠️ WEAK POSITIVE CORRELATION")
    else:
        print(f"\n   ❌ NO CORRELATION - signals may not compound")


📊 SIGNAL COUNT CORRELATION

📈 HIGH SIGNAL (3+ signals):
   Trades: 100
   Win rate: 62%
   Avg gain: 17.8%

📉 LOW SIGNAL (1-2 signals):
   Trades: 448
   Win rate: 45%
   Avg gain: 7.1%

📊 CORRELATIONS:
   Signal count → Win rate: 0.13
   Signal count → Gain: 0.28

   ⚠️ WEAK POSITIVE CORRELATION


In [9]:
# SIMULATED TRADING RESULTS
print("\n" + "="*60)
print("💰 SIMULATED TRADING RESULTS")
print("="*60)

if len(df) > 0:
    # Simulate trading with different strategies
    starting_capital = 10000
    
    # Strategy 1: Trade ALL setups
    all_trades_return = df['best_gain'].sum()
    
    # Strategy 2: Trade only 3+ signal setups
    high_signal_return = high_signal['best_gain'].sum() if len(high_signal) > 0 else 0
    
    # Strategy 3: Trade only 4 signal setups (all aligned)
    perfect_signal = df[df['signal_count'] == 4]
    perfect_return = perfect_signal['best_gain'].sum() if len(perfect_signal) > 0 else 0
    
    print(f"\n💼 STRATEGY COMPARISON (6 months):")
    print("-" * 50)
    print(f"\nStarting capital: ${starting_capital:,}")
    
    print(f"\n📊 Strategy 1: Trade ALL setups")
    print(f"   Trades: {len(df)}")
    print(f"   Total return: {all_trades_return:.1f}%")
    print(f"   Final value: ${starting_capital * (1 + all_trades_return/100):,.0f}")
    
    print(f"\n📊 Strategy 2: Trade 3+ signals only")
    print(f"   Trades: {len(high_signal)}")
    print(f"   Total return: {high_signal_return:.1f}%")
    print(f"   Final value: ${starting_capital * (1 + high_signal_return/100):,.0f}")
    
    print(f"\n📊 Strategy 3: Trade 4 signals (perfect setup)")
    print(f"   Trades: {len(perfect_signal)}")
    print(f"   Total return: {perfect_return:.1f}%")
    print(f"   Final value: ${starting_capital * (1 + perfect_return/100):,.0f}")


💰 SIMULATED TRADING RESULTS

💼 STRATEGY COMPARISON (6 months):
--------------------------------------------------

Starting capital: $10,000

📊 Strategy 1: Trade ALL setups
   Trades: 548
   Total return: 4943.6%
   Final value: $504,358

📊 Strategy 2: Trade 3+ signals only
   Trades: 100
   Total return: 1785.0%
   Final value: $188,499

📊 Strategy 3: Trade 4 signals (perfect setup)
   Trades: 3
   Total return: 71.0%
   Final value: $17,099


In [10]:
# TOP OPPORTUNITIES NOW
print("\n" + "="*60)
print("🎯 RECENT/CURRENT OPPORTUNITIES")
print("="*60)

if len(df) > 0:
    # Get recent setups (last 30 days)
    recent = df[df['entry_date'] >= (datetime.now() - timedelta(days=30))]
    recent = recent.sort_values('signal_count', ascending=False)
    
    print(f"\n📊 Last 30 days setups (top 10):")
    print(f"\n{'Ticker':<8} {'Date':<12} {'Signals':>8} {'Gain':>8} {'Result'}")
    print("-" * 50)
    
    for _, row in recent.head(10).iterrows():
        date_str = row['entry_date'].strftime('%m/%d')
        result = "✅ WIN" if row['is_winner'] else "❌ LOSS"
        print(f"{row['ticker']:<8} {date_str:<12} {row['signal_count']:>8} {row['best_gain']:>7.1f}% {result}")


🎯 RECENT/CURRENT OPPORTUNITIES


TypeError: Invalid comparison between dtype=datetime64[ns, America/New_York] and datetime

In [11]:
# FINAL THESIS VERDICT
print("\n" + "="*80)
print("🎯 FINAL THESIS VERDICT: COMBINED SYSTEM")
print("="*80)

if len(df) > 10:
    # Key metrics
    overall_win_rate = df['is_winner'].mean() * 100
    overall_avg_gain = df['best_gain'].mean()
    
    high_signal_win = high_signal['is_winner'].mean() * 100 if len(high_signal) > 0 else 0
    high_signal_gain = high_signal['best_gain'].mean() if len(high_signal) > 0 else 0
    
    print(f"\n📊 SUMMARY:")
    print("-" * 40)
    print(f"   Serial runners found: {len(serial_runners)}")
    print(f"   Total trade setups: {len(df)}")
    print(f"   High signal setups (3+): {len(high_signal)}")
    print(f"   Perfect setups (4): {len(df[df['signal_count']==4])}")
    
    print(f"\n📈 PERFORMANCE:")
    print("-" * 40)
    print(f"   Overall win rate: {overall_win_rate:.0f}%")
    print(f"   Overall avg gain: {overall_avg_gain:.1f}%")
    print(f"   High signal win rate: {high_signal_win:.0f}%")
    print(f"   High signal avg gain: {high_signal_gain:.1f}%")
    
    # VERDICT
    print(f"\n" + "="*40)
    
    if high_signal_win >= 65 and high_signal_gain >= 10:
        print(f"✅ THESIS VALIDATED: THE GAME IS RIGGED (in our favor)")
        print(f"\n🎯 THE EDGE IS REAL:")
        print(f"   - Serial runners exist and are identifiable")
        print(f"   - Day 0 setup signals are detectable")
        print(f"   - Combining signals improves win rate to {high_signal_win:.0f}%")
        print(f"   - Average gain of {high_signal_gain:.1f}% per trade")
        
        print(f"\n📋 THE SYSTEM:")
        print(f"   1. Focus on serial runners ({', '.join(list(serial_runners.keys())[:5])}, ...)")
        print(f"   2. Wait for Day 0 setup (rest day with low volume)")
        print(f"   3. Check sector leader for exhaustion")
        print(f"   4. Enter on Day 1 with gap+volume confirmation")
        print(f"   5. Hold for 3-5 days (typical run length)")
        print(f"   6. Take profits at 15-20%+ or on first red day")
        
    elif overall_win_rate >= 55 and overall_avg_gain >= 5:
        print(f"⚠️ THESIS PARTIALLY VALIDATED")
        print(f"   Edge exists but is smaller than hoped")
        print(f"   Win rate: {overall_win_rate:.0f}%")
        print(f"   Avg gain: {overall_avg_gain:.1f}%")
        print(f"   Continue refining signals")
        
    else:
        print(f"❌ THESIS KILLED: Edge not tradeable")
        print(f"   Win rate: {overall_win_rate:.0f}% (need 60%+)")
        print(f"   Avg gain: {overall_avg_gain:.1f}% (need 10%+)")
        print(f"   Either patterns don't exist or signals don't work")

else:
    print(f"\n⚠️ INSUFFICIENT DATA")
    print(f"   Only {len(df)} trade setups found")
    print(f"   Need more data to validate/kill thesis")

print(f"\n" + "="*80)
print("🐺 BROKKR ANALYSIS COMPLETE")
print("="*80)


🎯 FINAL THESIS VERDICT: COMBINED SYSTEM

📊 SUMMARY:
----------------------------------------
   Serial runners found: 19
   Total trade setups: 548
   High signal setups (3+): 100
   Perfect setups (4): 3

📈 PERFORMANCE:
----------------------------------------
   Overall win rate: 48%
   Overall avg gain: 9.0%
   High signal win rate: 62%
   High signal avg gain: 17.8%

❌ THESIS KILLED: Edge not tradeable
   Win rate: 48% (need 60%+)
   Avg gain: 9.0% (need 10%+)
   Either patterns don't exist or signals don't work

🐺 BROKKR ANALYSIS COMPLETE
